## Elliptic Curves

In [1]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.asymmetric.utils import decode_dss_signature
from cryptography.hazmat.primitives.serialization import (
    Encoding, ParameterFormat, PublicFormat,
    load_pem_parameters, load_pem_public_key
)

## ECDSA

ECDSA is a Digital Signature Algorithm. 

We use NIST curve SECP384r1. The key size is 384 bits.
There is no need to generate parameters, because the standard already provides the curve equation, a generator and its order.

We proceed with the generation of a private key and of the corresponding public key.

In [2]:
private_key = ec.generate_private_key( ec.SECP384R1() )# precomputed curve
print("Private key is: %d" % private_key.private_numbers().private_value)# private key - int in [0, order]

Private key is: 19247871699885200742221632137670209853809385049703980534742023400444596567752941680102241827796718446513207493497864


In [ ]:
# Alternatively we can generate a key from an integer number (eg 1234) by using
# private_key = ec.derive_private_key( 1234, ec.SECP384R1() ) # provide your own number. Good not to store everything random

In [3]:
public_key = private_key.public_key() # point with 2 coodrinates. Can be verified it belongs to the curve
print("Public key is : (x = %s, y = %s)" % (public_key.public_numbers().x,public_key.public_numbers().y))

Public key is : (x = 15203351128299222528567889175835999860916913697209058137404867626405670645842333052321062351560357368804986968037752, y = 6126677711471697947955203283476180994852172108939647697378332648713716818233981584162154783778441127999668980056809)


Remember that the private key is an integer, while the public key is a point. Now we can sign a message.
The signature will be encoded using the DER rules, so it will be a string of bits.

In [7]:
data = b"this is some data I'd like to sign"#array of bytes
signature = private_key.sign(
    data,
    ec.ECDSA(hashes.SHA256())#hash functions
    )
#print(signature.hex())
print(signature)

b'0d\x020G\x19\xc8\xa3\x85\x1b\x1f\xf7\x81\x9bS\xa6W\xd1\xbe\xefu\x89\xe8\xf8\xebP\xdb>\xa5 \x01\xc1\xb2Q\x98&\xf8\x8a\n\xe5&j\xadJ7\xfc\xad\x9f\x89`O?\x020\x02\xa29pB\xfe\x113\xc5]`\xa70\x83\xc1\x9d^m\xea\x16jl\xae{\x04\x11\t\xd36Yt\xb4\xe6\xe4sMa\x18\x1f\xaf\x92\x8c\x16\x9e\xffl\xe5\x11'


The DSA signature is randomized, so if we run the algorithm again we get a different signature. The interface does not ask for a nonce, so we cannot make the mistake of providing repeated nonces.

In [8]:
signature = private_key.sign(
    data,
    ec.ECDSA(hashes.SHA256())
    )
#print(signature.hex())
print(signature)

b"0d\x020M\xb4R\xa51\x97xC\x8b\xddX<\x07(w\x81\xf0Z\x88\x172\xed\xe8\xe6\xb5\xea\xd2s\x8d\x1b{\x98\x9d\t\xdfe\xf4:D\xc8pc'\x07\xee{(\xb9\x020'\xd0\xf9&\xe3\xd6\xab&\x87\x89\xed\xacx\xd7$JO\xeb\xc9\xf1\xf3K5\xcd\x85;i\xee<\x07\xc1\x873\x8fn\x80\xbeLU-V\x10r\xf1\x85\x8d\xc3+"


We can decode the signature and retrieve the parameters $r$ and $s$

In [9]:
(r,s)=decode_dss_signature(signature)
print("r = %d\ns = %d" % (r,s) )

r = 11959799614451532424152901725618649100224261981710458319724022381395289787848572714581783146929493417497669117094073
s = 6128289720809215905962208042106446741458331579084670713789499687391736407717188313376675316944944261364225649001259


We can verify the signature using the public key. If the signature is wrong, we get an exception

In [10]:
try:
    public_key.verify(signature,data,ec.ECDSA(hashes.SHA256()))
    print("OK")
except:
    print("KO")

OK


## Lab

Find a classmate. One plays as Alice, the other plays as Bob.
Alice and Bob generate their keys from secret numbers and have published the corresponding public key.

Alice must prove to Bob that she is Alice.
She derives her private key from her secret number and signs a message.
Then she sends to Bob the message and the signature.
Bob verifies the signature using Alice's public key.

Bob proves his identity to Alice in the same way.

In [ ]:
# Alice
Alice_secret_number = 1234
Alice_serialized_public_key = b'-----BEGIN PUBLIC KEY-----\nMHYwEAYHKoZIzj0CAQYFK4EEACIDYgAEe2FShOUqdr8UOfiRMoGoEdS7PY4quCzZ\nprCsAxK1ft1/zoRprQ4Jsxul09T4aB2hG32SVNtusdPAn8ziDFnNH4rCtVJedReL\nURy3R3RY1EqiZ/y6ZLLo7NWdnbXWvQ7C\n-----END PUBLIC KEY-----\n'
Alice_message = b"I'm Alice"
# I will give you a signed by me your message->give you the sign->you verify using my public key

# You can load the other party's public key with
# remote_public_key = load_pem_public_key( serialized_public_key )

In [ ]:
# Bob
Bob_secret_number = 4321
Bob_serialized_public_key = b'-----BEGIN PUBLIC KEY-----\nMHYwEAYHKoZIzj0CAQYFK4EEACIDYgAENyisOoDb9lce4qW9DP+LueJUDXeCm89Y\n5ZYW0sx50UdwMwgWWk+HMNfct+tBW9AY2jqMRCPJgyIu9BucU38JuwchT6gsZrQi\nRJTXpndZMbKHD6j31yyaSZ2j9nGxNXkO\n-----END PUBLIC KEY-----\n'
Bob_message = b"I'm Bob"

# You can load the other party's public key with
# remote_public_key = load_pem_public_key( serialized_public_key )